# Day 9: Rope Bridge

This rope bridge creaks as you walk along it. You aren't sure how old it is, or whether it can even support your weight.

It seems to support the Elves just fine, though. The bridge spans a gorge which was carved out by the massive river far below you.

You step carefully; as you do, the ropes stretch and twist. You decide to distract yourself by modeling rope physics; maybe you can even figure out where not to step.

Consider a rope with a knot at each end; these knots mark the head and the tail of the rope. If the head moves far enough away from the tail, the tail is pulled toward the head.

Due to nebulous reasoning involving Planck lengths, you should be able to model the positions of the knots on a two-dimensional grid. Then, by following a hypothetical series of motions (your puzzle input) for the head, you can determine how the tail will move.

Due to the aforementioned Planck lengths, the rope must be quite short; in fact, the head (H) and tail (T) must always be touching (diagonally adjacent and even overlapping both count as touching):

```
....
.TH.
....

....
.H..
..T.
....

...
.H. (H covers T)
...
```

If the head is ever two steps directly up, down, left, or right from the tail, the tail must also move one step in that direction so it remains close enough:

```
.....    .....    .....
.TH.. -> .T.H. -> ..TH.
.....    .....    .....

...    ...    ...
.T.    .T.    ...
.H. -> ... -> .T.
...    .H.    .H.
...    ...    ...
```

Otherwise, if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up:

```
.....    .....    .....
.....    ..H..    ..H..
..H.. -> ..... -> ..T..
.T...    .T...    .....
.....    .....    .....

.....    .....    .....
.....    .....    .....
..H.. -> ...H. -> ..TH.
.T...    .T...    .....
.....    .....    .....
```

You just need to work out where the tail goes as the head follows a series of motions. Assume the head and the tail both start at the same position, overlapping.

For example:

```
R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2
```

This series of motions moves the head right four steps, then up four steps, then left three steps, then down one step, and so on. After each step, you'll need to update the position of the tail if the step means the head is no longer adjacent to the tail. Visually, these motions occur as follows (s marks the starting position as a reference point):

```
== Initial State ==

......
......
......
......
H.....  (H covers T, s)

== R 4 ==

......
......
......
......
TH....  (T covers s)

......
......
......
......
sTH...

......
......
......
......
s.TH..

......
......
......
......
s..TH.

== U 4 ==

......
......
......
....H.
s..T..

......
......
....H.
....T.
s.....

......
....H.
....T.
......
s.....

....H.
....T.
......
......
s.....

== L 3 ==

...H..
....T.
......
......
s.....

..HT..
......
......
......
s.....

.HT...
......
......
......
s.....

== D 1 ==

..T...
.H....
......
......
s.....

== R 4 ==

..T...
..H...
......
......
s.....

..T...
...H..
......
......
s.....

......
...TH.
......
......
s.....

......
....TH
......
......
s.....

== D 1 ==

......
....T.
.....H
......
s.....

== L 5 ==

......
....T.
....H.
......
s.....

......
....T.
...H..
......
s.....

......
......
..HT..
......
s.....

......
......
.HT...
......
s.....

......
......
HT....
......
s.....

== R 2 ==

......
......
.H....  (H covers T)
......
s.....

......
......
.TH...
......
s.....

After simulating the rope, you can count up all of the positions the tail visited at least once. In this diagram, s again marks the starting position (which the tail also visited) and # marks other positions the tail visited:

```
..##..
...##.
.####.
....#.
s###..
```

So, there are 13 positions the tail visited at least once.

Simulate your complete hypothetical series of motions. How many positions does the tail of the rope visit at least once?

In [1]:
#@title import file function

def import_directions(filename : str):
    directions = []
    with open(filename) as f:
        for line in f:
            direction, steps = line.rstrip('\n').split(' ')
            directions.append([direction, int(steps)])
    
    return directions

In [2]:
# @title test file import

import unittest

class TestProcessInput(unittest.TestCase):

    def test_import_directions(self):
        expected = [
            ['R', 2],
            ['D', 2],
            ['R', 2],
            ['U', 1],
            ['R', 2],
            ['L', 2],
            ['D', 1],
        ]
        self.assertEqual(import_directions('test.txt'), expected)

unittest.main(argv=[''], verbosity=2, exit=False)

test_import_directions (__main__.TestProcessInput.test_import_directions) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [7]:
def follow_directions(directions):
    tail_curr_x=0
    tail_curr_y=0
    head_curr_x=0
    head_curr_y=0
    # set containing all tail locations in format 'x,y'
    positions_tail_has_been=set() 
    for direction in directions:
        positions_tail_has_been.add(f'{tail_curr_x},{tail_curr_y}')
        for _ in range(direction[1]):
            match direction[0]:
                case 'U':
                    head_curr_y +=  1
                    if head_curr_y > tail_curr_y + 1:
                        tail_curr_y = head_curr_y - 1
                        tail_curr_x = head_curr_x
                    positions_tail_has_been.add(f'{tail_curr_x},{tail_curr_y}')
                case 'D':
                    head_curr_y -=  1
                    if head_curr_y < tail_curr_y - 1:
                        tail_curr_y = head_curr_y + 1
                        tail_curr_x = head_curr_x
                    positions_tail_has_been.add(f'{tail_curr_x},{tail_curr_y}')
                case 'R':
                    head_curr_x +=  1
                    if head_curr_x > tail_curr_x + 1:
                        tail_curr_x = head_curr_x - 1
                        tail_curr_y = head_curr_y
                    positions_tail_has_been.add(f'{tail_curr_x},{tail_curr_y}')
                case 'L':
                    head_curr_x -=  1
                    if head_curr_x < tail_curr_x - 1:
                        tail_curr_x = head_curr_x + 1
                        tail_curr_y = head_curr_y
                    positions_tail_has_been.add(f'{tail_curr_x},{tail_curr_y}')
            print(f'head: {head_curr_x},{head_curr_y} tail: {tail_curr_x},{tail_curr_y}') 
            print(f'positions_tail_has_been: {positions_tail_has_been}') 
            
    return len(positions_tail_has_been)    

In [8]:
# @title test follow_directions

import unittest

class TestProcessDirections(unittest.TestCase):

    def test_follow_directions(self):
        input = [
            ['R', 4],
            ['U', 4],
            ['L', 3],
            ['D', 1],
            ['R', 4],
            ['D', 1],
            ['L', 5],
            ['R', 2],
        ]
        self.assertEqual(follow_directions(input), 13)

unittest.main(argv=[''], verbosity=2, exit=False)

test_follow_directions (__main__.TestProcessDirections.test_follow_directions) ... ok
test_import_directions (__main__.TestProcessInput.test_import_directions) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.005s

OK


head: 1,0 tail: 0,0
positions_tail_has_been: {'0,0'}
head: 2,0 tail: 1,0
positions_tail_has_been: {'1,0', '0,0'}
head: 3,0 tail: 2,0
positions_tail_has_been: {'1,0', '0,0', '2,0'}
head: 4,0 tail: 3,0
positions_tail_has_been: {'1,0', '3,0', '0,0', '2,0'}
head: 4,1 tail: 3,0
positions_tail_has_been: {'1,0', '3,0', '0,0', '2,0'}
head: 4,2 tail: 4,1
positions_tail_has_been: {'0,0', '4,1', '3,0', '1,0', '2,0'}
head: 4,3 tail: 4,2
positions_tail_has_been: {'4,2', '0,0', '4,1', '3,0', '1,0', '2,0'}
head: 4,4 tail: 4,3
positions_tail_has_been: {'4,2', '0,0', '4,1', '3,0', '4,3', '1,0', '2,0'}
head: 3,4 tail: 4,3
positions_tail_has_been: {'4,2', '0,0', '4,1', '3,0', '4,3', '1,0', '2,0'}
head: 2,4 tail: 3,4
positions_tail_has_been: {'4,2', '0,0', '4,1', '3,0', '3,4', '4,3', '1,0', '2,0'}
head: 1,4 tail: 2,4
positions_tail_has_been: {'4,2', '2,4', '0,0', '4,1', '3,0', '3,4', '4,3', '1,0', '2,0'}
head: 1,3 tail: 2,4
positions_tail_has_been: {'4,2', '2,4', '0,0', '4,1', '3,0', '3,4', '4,3', '1,0', 